In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat
from scipy import signal

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.metrics import classification_report

In [4]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/SPH"
print(os.listdir(data_dir))

['metadata.csv', 'data_df.csv', 'data_df_no1.csv', 'records']


In [3]:
main_df = pd.read_csv(data_dir + "/data_df.csv")
main_df.shape

(20792, 4)

In [4]:
single_fns = main_df["File name"].values.tolist()
single_mat_paths = [data_dir + f"/records/{x}.h5" for x in single_fns]

In [5]:
ratio = [0.8, 0.1]

train_index = int(len(single_mat_paths)*ratio[0])
valid_index = int(len(single_mat_paths)*(ratio[0]+ratio[1]))

train_mat_paths = single_mat_paths[:train_index]
valid_mat_paths = single_mat_paths[valid_index:]

train_label = main_df
valid_label = main_df

In [6]:
import h5py

class HeartData(Dataset):
    def __init__(self, data_paths):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]
        a = h5py.File(data_path, 'r')
        data_h5 = a['ecg']
        data = np.array(data_h5)
        clip_data = data[:, :5000]
        z_normalized = (clip_data - clip_data.mean()) / (clip_data.std())      
        f = signal.resample(clip_data, 2500)

        filename = data_path.split("/")[-1].split(".")[0]
        label = main_df[main_df["File name"] == filename]["New Label"].values.item()

        torch_data = torch.from_numpy(clip_data)

        return torch_data.float(), label

    def __len__(self):
        return len(self.data_paths)

In [7]:
train_ds = HeartData(train_mat_paths)
valid_ds = HeartData(valid_mat_paths)

In [20]:
class Teacher_Network(nn.Module):
    def __init__(self):
        super().__init__()

        # DownBlock 1
        self.down_block11 = nn.Sequential(
            nn.Conv1d(12, 64, 3, 1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        self.down_block12 = nn.Sequential(
            nn.Conv1d(64, 64, 3, 1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        self.mp = nn.MaxPool1d(2, stride=2)
    
        # DownBlock 2
        self.down_block21 = nn.Sequential(
            nn.Conv1d(64, 128, 3, 1, 2),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )

        self.down_block22 = nn.Sequential(
            nn.Conv1d(128, 128, 3, 1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        # DownBlock 3
        self.down_block31 = nn.Sequential(
            nn.Conv1d(128, 256, 3, 1, 2),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )

        self.down_block32 = nn.Sequential(
            nn.Conv1d(256, 256, 3, 1),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
        
        # DownBlock 4
        self.down_block41 = nn.Sequential(
            nn.Conv1d(256, 512, 3, 1, 2),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )

        self.down_block42 = nn.Sequential(
            nn.Conv1d(512, 512, 3, 1),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )

        # DownBlock 5
        self.down_block51 = nn.Sequential(
            nn.Conv1d(512, 1024, 3, 1, 2),
            nn.BatchNorm1d(1024),
            nn.ReLU()
        )

        self.down_block52 = nn.Sequential(
            nn.Conv1d(1024, 1024, 3, 1),
            nn.BatchNorm1d(1024),
            nn.ReLU()
        )

        # UpBlock 1
        self.up_block11 = nn.Upsample(scale_factor=2)
        self.up_block12 = nn.ConvTranspose1d(1024, 512, 3, 1, 1)

        self.relu1 = nn.ReLU()
        self.output1 = nn.Sequential(
            nn.Conv1d(512,512,1,1),
            nn.Sigmoid()
        )
        
        self.up_block13 = nn.Sequential(
            nn.Conv1d(512, 512, 3, 1,1),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )

        # UpBlock 2
        self.up_block21 = nn.Upsample(scale_factor=2)
        self.up_block22 = nn.ConvTranspose1d(512, 256, 3, 1, 1)

        self.relu2 = nn.ReLU()
        self.output2 = nn.Sequential(
            nn.Conv1d(256,256,1,1),
            nn.Sigmoid()
        )

        self.up_block23 = nn.Sequential(
            nn.Conv1d(256, 256, 3, 1, 1),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )

        # UpBlock 3
        self.up_block31 = nn.Upsample(scale_factor=2)
        self.up_block32 = nn.ConvTranspose1d(256, 128, 3, 1, 1)

        self.relu3 = nn.ReLU()
        self.output3 = nn.Sequential(
            nn.Conv1d(128,128,1,1),
            nn.Sigmoid()
        )

        self.up_block33 = nn.Sequential(
            nn.Conv1d(128, 128, 3, 1,1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )

        # UpBlock 4
        self.up_block41 = nn.Upsample(scale_factor=2)
        self.up_block42 = nn.ConvTranspose1d(128, 64, 3, 1, 1)

        self.relu4 = nn.ReLU()
        self.output4 = nn.Sequential(
            nn.Conv1d(64,64,1,1),
            nn.Sigmoid()
        )
        
        self.up_block43 = nn.Sequential(
            nn.Conv1d(64, 64, 3, 1,1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        # OutBlock
        self.out = nn.Conv1d(64, 3, 1)
    
    def forward(self, x):
        x11 = self.down_block11(x)
        x12 = self.down_block12(x11)
        x13 = self.mp(x12)

        # x11 = self.down_block11(x)
        # x12 = self.down_block12(x11)
        # x13 = self.mp(x11+x12)
        
        x21 = self.down_block21(x13)
        x22 = self.down_block22(x21)
        x23 = self.mp(x22)
        
        x31 = self.down_block31(x23)
        x32 = self.down_block32(x31)
        x33 = self.mp(x32)
        
        x41 = self.down_block41(x33)
        x42 = self.down_block42(x41)
        x43 = self.mp(x42)
        
        x5 = self.down_block51(x43)
        x5 = self.down_block52(x5)

        x61 = self.up_block11(x5)
        x62 = self.up_block12(x61)
        x6 = self.relu1(x42 + x62)
        x6 = self.output1(x6)
        x6_att = x6 * x62
        x6 = self.up_block13(x6_att)
        x6 = self.up_block13(x6)
        
        x71 = self.up_block21(x6)
        x72 = self.up_block22(x71)
        x7 = self.relu2(x32 + x72)
        x7 = self.output2(x7)
        x7_att = x7 * x72
        x7 = self.up_block23(x7_att)
        x7 = self.up_block23(x7)

        x81 = self.up_block31(x7)
        x82 = self.up_block32(x81)
        x8 = self.relu3(x22 + x82)
        x8 = self.output3(x8)
        x8_att = x8 * x82
        x8 = self.up_block33(x8_att)
        x8 = self.up_block33(x8)

        x91 = self.up_block41(x8)
        x92 = self.up_block42(x91)
        x9 = self.relu4(x12 + x92)
        x9 = self.output4(x9)
        x9_att = x9 * x92
        x9 = self.up_block43(x9_att)
        x9 = self.up_block43(x9)

        x10 = self.out(x9)
        
        return x7_att, x8_att, x9_att

In [21]:
test1 = torch.randn(1, 12, 2500)
model1 = Teacher_Network()
a = model1(test1)
print(a.shape)

AttributeError: 'tuple' object has no attribute 'shape'

In [36]:
loss_CE = torch.nn.CrossEntropyLoss()

In [3]:
# from torch import nn, einsum
# from einops import rearrange, repeat

def exists(val):
    return val is not None

def default(val, d):
    if exists(val):
        return val
    return d() if callable(d) else d

class Identity(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

    def forward(self, x, *args, **kwargs):
        return x


class LayerNorm(nn.Module):
    def __init__(self, feats, stable = False, dim = -1):
        super().__init__()
        self.stable = stable
        self.dim = dim

        self.g = nn.Parameter(torch.ones(feats, *((1,) * (-dim - 1))))

    def forward(self, x):
        dtype, dim = x.dtype, self.dim

        if self.stable:
            x = x / x.amax(dim = dim, keepdim = True).detach()

        eps = 1e-5 if x.dtype == torch.float32 else 1e-3
        var = torch.var(x, dim = dim, unbiased = False, keepdim = True)
        mean = torch.mean(x, dim = dim, keepdim = True)

        return (x - mean) * (var + eps).rsqrt().type(dtype) * self.g.type(dtype)


class CrossAttention(nn.Module):
    def __init__(
        self,
        dim,
        *,
        context_dim = None,
        dim_head = 64,
        heads = 8,
        norm_context = False,
        scale = 8
    ):
        super().__init__()
        self.scale = scale

        self.heads = heads
        inner_dim = dim_head * heads

        context_dim = default(context_dim, dim)

        self.norm = LayerNorm(dim)
        self.norm_context = LayerNorm(context_dim) if norm_context else Identity()

        self.null_kv = nn.Parameter(torch.randn(2, dim_head))
        self.to_q = nn.Linear(dim, inner_dim, bias = False)
        self.to_kv = nn.Linear(context_dim, inner_dim * 2, bias = False)

        self.q_scale = nn.Parameter(torch.ones(dim_head))
        self.k_scale = nn.Parameter(torch.ones(dim_head))

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim, bias = False),
            LayerNorm(dim)
        )

    def forward(self, x, context, mask = None): 
        b, n, device = *x.shape[:2], x.device

        x = self.norm(x)
        context = self.norm_context(context)

        q, k, v = (self.to_q(x), *self.to_kv(context).chunk(2, dim = -1))

        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), (q, k, v))

        # add null key / value for classifier free guidance in prior net

        nk, nv = map(lambda t: repeat(t, 'd -> b h 1 d', h = self.heads,  b = b), self.null_kv.unbind(dim = -2))

        k = torch.cat((nk, k), dim = -2)
        v = torch.cat((nv, v), dim = -2)

        # cosine sim attention

        q, k = map(l2norm, (q, k))
        q = q * self.q_scale
        k = k * self.k_scale

        # similarities

        sim = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale

        # masking

        max_neg_value = -torch.finfo(sim.dtype).max

        if exists(mask):
            mask = F.pad(mask, (1, 0), value = True)
            mask = rearrange(mask, 'b j -> b 1 1 j')
            sim = sim.masked_fill(~mask, max_neg_value)

        attn = sim.softmax(dim = -1, dtype = torch.float32)
        attn = attn.to(sim.dtype)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

ModuleNotFoundError: No module named 'einops'

In [74]:
class Student_Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Start
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(12, 64, 3, 1, 0, 2),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        
        # ConvBlock 1
        self.conv_block11 = nn.Sequential(
            nn.Conv1d(64, 64, 3, 1, 1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
    
        self.conv_block12 = nn.Sequential(
            nn.Conv1d(64, 64, 3, 1, 1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        
        # ConvBlock 2
        self.conv_block21 = nn.Sequential(
            nn.Conv1d(64, 128, 3, 1, 2, 2),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
    
        self.conv_block22 = nn.Sequential(
            nn.Conv1d(128, 128, 3, 1, 1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
    
        # Attention 1
        self.att_block11 = nn.Sequential(
            nn.Conv1d(64, 64, 1, 1),
            nn.BatchNorm1d(64)
        )
    
        # self.att_block12 = nn.Sequential(
        #     nn.Conv1d(128, 128, 1, 1),
        #     nn.BatchNorm1d(128)
        # )
            
        self.att_block13 = nn.Sequential(
            nn.ReLU(),
            nn.Conv1d(64, 128, 3, 1, 1),
            nn.BatchNorm1d(128),
            nn.Sigmoid()
        )

        self.att_block12 = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ConvTranspose1d(128, 64, 3, 1, 1),
            nn.BatchNorm1d(64)
        )
        
        # ConvBlock 3
        self.conv_block31 = nn.Sequential(
            nn.Conv1d(128, 256, 3, 1),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
    
        self.conv_block32 = nn.Sequential(
            nn.Conv1d(256, 256, 3, 1),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
    
        # Attention 2
        self.att_block21 = nn.Sequential(
            nn.Conv1d(128, 256, 1, 1),
            nn.BatchNorm1d(256)
        )
    
        self.att_block22 = nn.Sequential(
            nn.Conv1d(256, 256, 1, 1),
            nn.BatchNorm1d(256)
        )
            
        self.att_block23 = nn.Sequential(
            nn.ReLU(),
            nn.Conv1d(256, 256, 3, 1),
            nn.BatchNorm1d(256),
            nn.Sigmoid()
        )
    
        # ConvBlock 4
        self.conv_block41 = nn.Sequential(
            nn.Conv1d(256, 512, 3, 1),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
    
        self.conv_block42 = nn.Sequential(
            nn.Conv1d(512, 512, 3, 1),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
    
        # Attention 3
        self.att_block31 = nn.Sequential(
            nn.Conv1d(256, 512, 1, 1),
            nn.BatchNorm1d(512)
        )
    
        self.att_block32 = nn.Sequential(
            nn.Conv1d(512, 512, 1, 1),
            nn.BatchNorm1d(512)
        )
  
        self.att_block33 = nn.Sequential(
            nn.ReLU(),
            nn.Conv1d(512, 512, 3, 1),
            nn.BatchNorm1d(512),
            nn.Sigmoid()
        )

        self.mp = nn.MaxPool1d(2, stride=2)
        # self.linear = nn.Linear(1)

        # Out
        self.out = nn.Sequential(
            nn.AdaptiveAvgPool1d(512),
            nn.Linear(512,31)
        )
    
    def forward(self, x):
        x1 = self.conv_block1(x)
        x1 = self.conv_block11(x1)
        x1 = self.conv_block12(x1)
        x1 = self.conv_block11(x1)
        x1 = self.conv_block12(x1)
        x1 = self.conv_block11(x1)
        x1 = self.conv_block12(x1)

        x2 = self.conv_block21(x1)
        # print(x2.shape)
        x2 = self.conv_block22(x2)
        # print(x2.shape)
        x2 = self.conv_block22(x2)
        x2 = self.conv_block22(x2)
        x2 = self.conv_block22(x2)
        x2 = self.conv_block22(x2)
        x2 = self.conv_block22(x2)
        x2 = self.conv_block22(x2)
        # print(x2.shape)
        x2 = self.mp(x2)
        print(x2.shape)

        x_att11 = self.att_block11(x1)
        # x_att12 = self.att_block12(x2)
        x_att12 = self.att_block12(x2)
        # print(x_att11.shape)
        # print(x_att12.shape)
        x_att1 = x_att11 + x_att12
        x_att1 = self.att_block13(x_att1)
        x_att1 = self.mp(x_att1)
        print(x_att1.shape)
        x_att1 = x_att1 * x2
        # print(x_att1.shape)

        x3 = self.conv_block31(x_att1)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)
        x3 = self.conv_block32(x3)

        x_att21 = self.att_block21(x2)
        x_att22 = self.att_block22(x3)
        x_att2 = x_att21 + x_att22
        x_att2 = self.att_block23(x_att2)
        x_att2 = x_att2 * x3

        x4 = self.conv_block41(x_att2)
        x4 = self.conv_block42(x4)
        x4 = self.conv_block42(x4)
        x4 = self.conv_block42(x4)
        x4 = self.conv_block42(x4)
        x4 = self.conv_block42(x4)

        x_att31 = self.att_block31(x3)
        x_att32 = self.att_block32(x4)
        x_att3 = x_att31 + x_att32
        x_att3 = self.att_block33(x_att3)
        x_att3 = x_att13 * x4

        x_out = self.out(x_att3)

        return x_out

In [75]:
test1 = torch.randn(1, 12, 2500)
model1 = Student_Network()
a = model1(test1)
print(a.shape)

torch.Size([1, 128, 1248])
torch.Size([1, 128, 1248])


RuntimeError: The size of tensor a (1248) must match the size of tensor b (1224) at non-singleton dimension 2